# Feature engineering & selection

In [18]:
import pandas as pd

TRAINING_SET_FILE = 'data/training_set.csv'

ori = pd.read_csv(TRAINING_SET_FILE, sep=';')
ori.T
df = ori

In [16]:

# convertimos a datetimes
date_columns = ['fecha_reserva', 'check-in', 'check-out']
for d in date_columns:
    df[d] = pd.to_datetime(df[d])

df['reservation_days_ago'] = (df['check-in'] - df['fecha_reserva']).dt.days
df['total_noches'] = (df['check-out'] - df['check-in']).dt.days
df['precio_por_noche'] = df['importe_reserva'] / df['roomnigths']

groupby_country = df.groupby(by='pais_cliente')['is_reserva_cancelada']
mean_country_cancelations = groupby_country.sum() / groupby_country.count()

mean_country_cancelations = pd.DataFrame(mean_country_cancelations)\
    .rename(columns={'is_reserva_cancelada': 'pct_cancelacion_pais'})
mean_country_cancelations.reset_index(level=0, inplace=True)

df = df.merge(mean_country_cancelations, left_on='pais_cliente', right_on='pais_cliente')

#### Porcen Reser vs Cancelacion por pais

In [25]:
res_por_pais = pd.pivot_table(ori,index=["pais_cliente"],values=["index"], aggfunc=lambda x: len(x.unique()))
can_por_pais = pd.pivot_table(ori,index=["pais_cliente"],values=["is_reserva_cancelada"], aggfunc=sum)
result = pd.concat([res_por_pais, can_por_pais], axis=1, join='inner',join_axes=[res_por_pais.index])
result["%"] = result["is_reserva_cancelada"]*100/result["index"]
result = result.sort_values("%", ascending=False)

In [ ]:
res_por_fecha_reserva = pd.pivot_table(df,index=["fecha_reserva"],values=["index"], aggfunc=lambda x: len(x.unique()))
can_por_fecha_reserva = pd.pivot_table(df,index=["fecha_reserva"],values=["is_reserva_cancelada"], aggfunc=sum)
result = pd.concat([res_por_pais, can_por_fecha_reserva], axis=1, join='inner',join_axes=[res_por_pais.index])
result["%"] = result["is_reserva_cancelada"]*100/result["index"]
result.sort_values("%", ascending=False)

# Feature selection

De todas las características que hemos construido, vamos a evaluar como funcionan.

## Date extractions

In [ ]:
df['fecha_reserva_weekday'] = df.fecha_reserva.dt.weekday
df['fecha_reserva_day'] = df.fecha_reserva.dt.day

df['checkin_weekday'] = df['check-in'].dt.weekday
df['checkin_month'] = df['check-in'].dt.month
df['checkin_day'] = df['check-in'].dt.day

df['checkout_weekday'] = df['check-out'].dt.weekday
df['checkout_month'] = df['check-out'].dt.month
df['checkout_day'] = df['check-in'].dt.day


In [ ]:
pd.get_dummies(df.fecha_reserva)

,index,is_reserva_cancelada,%
fecha_reserva,,,
2017-06-04,2394,813,33.959900
2017-06-05,3762,1198,31.844763
2017-06-01,3505,1115,31.811698
2017-06-03,2419,754,31.169905
2017-06-08,3843,1174,30.549050
2017-06-02,3497,1062,30.368888
2017-06-06,4038,1212,30.014859
2017-06-07,3910,1172,29.974425


In [21]:
ori.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27368 entries, 0 to 27367
Data columns (total 14 columns):
index                   27368 non-null int64
fecha_reserva           27368 non-null object
pais_cliente            27333 non-null object
importe_reserva         27368 non-null float64
roomnigths              27368 non-null int64
check-in                27368 non-null object
check-out               27368 non-null object
is_reserva_cancelada    27368 non-null int64
total_habitaciones      27368 non-null int64
hotel                   27368 non-null object
localizador             27368 non-null object
nombre_cliente          27368 non-null object
telefono_cliente        27368 non-null object
email_cliente           27368 non-null object
dtypes: float64(1), int64(4), object(9)
memory usage: 2.9+ MB
